# Week 3 - Project: Churn Prediction for  StreamWorks Media. 

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns  
import sklearn as sk

## 1. Load & Explore the Data 

In [ ]:
file_name = 'streamworks_user_data.csv'